In [3]:
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import logging
import matplotlib.pyplot as plt
import numpy as np

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.DEBUG)

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2

try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl
import holoviews as hv
bpl.output_notebook()

/home/qiushou/.conda/envs/caiman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/qiushou/.conda/envs/caiman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/qiushou/.conda/envs/caiman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/qiushou/.conda/envs/caiman/lib/python3.7/site-

Loading BokehJS ...

## Grab 2 videos from the file path

In [5]:
import glob
import re
import os
msFilePath='/home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14'
msFilename = sorted(glob.glob(os.path.join(msFilePath,'ms*.avi')))
filenum=[]
for file in msFilename:
    filenum.append(int(re.split(r'(\d+)',file.rsplit('/',-1)[-1].rsplit('.',1)[0])[1]))
msFileList=[msFilename[i] for i in np.argsort(filenum)]
msFileList=msFileList[:2]
msFileList

['/home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/msCam1.avi',
 '/home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/msCam2.avi']

## Crop and resize the selected videos

In [6]:
import moviepy.video as mpv
from moviepy.editor import *
import matplotlib as mpl
import cv2

# downsample raw videos by 0.5 width * 0.5 height
clip = VideoFileClip(msFileList[0])
im=clip.get_frame(1)
scale_percent = 50
#calculate the 50 percent of original dimensions
width = int(im.shape[1] * scale_percent / 100)
height = int(im.shape[0] * scale_percent / 100)
# dsize
dsize = (width, height)
# resize image
imds = cv2.resize(im, dsize)

r=cv2.selectROI(imds,fromCenter=False)
print(r)
x1=int(r[0])
x2=int(r[0]+r[2])
y1=int(r[1])
y2=int(r[1]+r[3])

imcrop=imds[y1:y2,x1:x2]
cv2.imshow('Image',imcrop)
cv2.waitKey(0)
cv2.destroyAllWindows()
print('Done')
np.shape(imcrop)

(51, 3, 265, 235)
Done


(235, 265, 3)

## Concatenate the videos

In [7]:
videoconcat=msFilePath + '/' + 'sample_concat.avi'
spatial_downsampling=2
temporal_downsampling=3
cropped_clip_list=[]
iframe=0
for video in msFileList:
    print('Concatenating '+video)
    clip = VideoFileClip(video)
    if spatial_downsampling!=1:
        clip=clip.resize(1/spatial_downsampling)
    cropped_clip=mpv.fx.all.crop(clip,x1=x1,y1=y1,x2=x2,y2=y2)
    
    cropped_clip_list.append(cropped_clip)
final_clip=concatenate_videoclips(cropped_clip_list)
if temporal_downsampling>1:
    final_clip=mpv.fx.all.speedx(final_clip, factor=temporal_downsampling)
final_clip.write_videofile(videoconcat,codec='rawvideo',audio=False)

t:   0%|          | 0/667 [00:00<?, ?it/s, now=None]

Concatenating /home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/msCam1.avi
Concatenating /home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/msCam2.avi
Moviepy - Building video /home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/sample_concat.avi.
Moviepy - Writing video /home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/sample_concat.avi



Moviepy - Done !
Moviepy - video ready /home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/sample_concat.avi


In [17]:
#Play the movie, press q to exit
fnames=[videoconcat]
m_orig = cm.load_movie_chain(fnames)
downsample_ratio = .5  # motion can be perceived better when downsampling in time
m_orig.resize(1, 1, downsample_ratio).play(q_max=99.5, fr=30, magnification=2)   # play movie (press q to exit)

100%|██████████| 1/1 [00:00<00:00, 10.02it/s]
    42678787 [movies.py:              resize():958] [14950] reshaping along z


In [18]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=vimdview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

    81847328 [motion_correction.py:tile_and_correct_wrapper():2922] [21403] 0
    81847329 [motion_correction.py:tile_and_correct_wrapper():2922] [21405] 0
    81847335 [motion_correction.py:tile_and_correct_wrapper():2922] [21404] 0
    81847335 [motion_correction.py:tile_and_correct_wrapper():2922] [21407] 0
    81847338 [motion_correction.py:tile_and_correct_wrapper():2922] [21408] 0
    81847334 [motion_correction.py:tile_and_correct_wrapper():2922] [21406] 0
    81847353 [motion_correction.py:tile_and_correct_wrapper():2922] [21409] 0
    81847352 [motion_correction.py:tile_and_correct_wrapper():2922] [21410] 0
    81849457 [motion_correction.py:tile_and_correct_wrapper():2922] [21405] 10
    81849450 [motion_correction.py:tile_and_correct_wrapper():2922] [21408] 10
    81849523 [motion_correction.py:tile_and_correct_wrapper():2922] [21407] 10
    81849595 [motion_correction.py:tile_and_correct_wrapper():2922] [21409] 10
    81849675 [motion_correction.py:tile_and_correct_wrapper(

    81958659 [motion_correction.py:tile_and_correct_wrapper():2922] [21409] 40
    81958969 [motion_correction.py:tile_and_correct_wrapper():2922] [21403] 40
    81958978 [motion_correction.py:tile_and_correct_wrapper():2922] [21407] 40
    81959186 [motion_correction.py:tile_and_correct_wrapper():2922] [21405] 40
    81959277 [motion_correction.py:tile_and_correct_wrapper():2922] [21406] 40
    81959616 [motion_correction.py:tile_and_correct_wrapper():2922] [21404] 40
    81959995 [motion_correction.py:tile_and_correct_wrapper():2922] [21410] 40
    81960348 [motion_correction.py:tile_and_correct_wrapper():2922] [21408] 40
    81960874 [motion_correction.py:tile_and_correct_wrapper():2922] [21409] 50
    81961207 [motion_correction.py:tile_and_correct_wrapper():2922] [21407] 50
    81961376 [motion_correction.py:tile_and_correct_wrapper():2922] [21403] 50
    81961401 [motion_correction.py:tile_and_correct_wrapper():2922] [21406] 50
    81961765 [motion_correction.py:tile_and_correct_

    82047042 [motion_correction.py:tile_and_correct_wrapper():2922] [21406] 80
    82047176 [motion_correction.py:tile_and_correct_wrapper():2922] [21405] 70
    82047838 [motion_correction.py:tile_and_correct_wrapper():2922] [21409] 80
    82047967 [motion_correction.py:tile_and_correct_wrapper():2922] [21408] 80
    82048137 [motion_correction.py:tile_and_correct_wrapper():2922] [21410] 80
    82048257 [motion_correction.py:tile_and_correct_wrapper():2922] [21403] 80
    82048565 [motion_correction.py:tile_and_correct_wrapper():2922] [21407] 80
    82049053 [motion_correction.py:tile_and_correct_wrapper():2922] [21405] 80


In [19]:
# dataset dependent parameters
fr = 10    # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds (GCamP6s)

# motion correction parameters
motion_correct = True            # flag for motion correction
pw_rigid = False                # flag for pw-rigid motion correction

gSig_filt = (8,8)   # size of filter, in general gSig (see below),
#                      change this one if algorithm does not work (Tuned by Qixin) # kernel of filter?
                    
max_shifts = (15,15)  # maximum allowed rigid shift
strides = (96,96)  # start a new patch for pw-rigid motion correction every x pixels
overlaps = (32,32)  # overlap between pathes (size of patch strides+overlaps)
# maximum deviation allowed for patch with respect to rigid shifts
max_deviation_rigid = 5
border_nan = 'copy'

mc_dict = {
    'fnames': fnames,
    'fr': fr,
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan
}

opts = params.CNMFParams(params_dict=mc_dict)


    81838763 [params.py:                 set():858] [14950] Changing key fnames in group data from None to ['/home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/sample_concat.avi']
    81838764 [params.py:                 set():858] [14950] Changing key fr in group data from 30 to 10
    81838765 [params.py:                 set():858] [14950] Changing key max_shifts in group motion from (6, 6) to (15, 15)
    81838765 [params.py:                 set():858] [14950] Changing key gSig_filt in group motion from None to (8, 8)
    81838766 [params.py:                 set():858] [14950] Changing key max_deviation_rigid in group motion from 3 to 5


In [22]:
if motion_correct:
    # do motion correction rigid
    mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
    mc.motion_correct(save_movie=True)
    fname_mc = mc.fname_tot_els if pw_rigid else mc.fname_tot_rig
    if pw_rigid:
        bord_px = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),
                                     np.max(np.abs(mc.y_shifts_els)))).astype(np.int)
    else:
        bord_px = np.ceil(np.max(np.abs(mc.shifts_rig))).astype(np.int)
        plt.subplot(1, 2, 1); plt.imshow(mc.total_template_rig)  # % plot template
        plt.subplot(1, 2, 2); plt.plot(mc.shifts_rig)  # % plot rigid shifts
        plt.legend(['x shifts', 'y shifts'])
        plt.xlabel('frames')
        plt.ylabel('pixels')

    bord_px = 0 if border_nan is 'copy' else bord_px
    fname_new = cm.save_memmap(fname_mc, base_name='memmap_', order='C',
                               border_to_0=bord_px)
else:  # if no motion correction just memory map the file
    fname_new = cm.save_memmap(filename_reorder, base_name='memmap_',
                               order='C', border_to_0=0, dview=dview)

print('Motion correction has been done!')

    82027915 [motion_correction.py:motion_correct_rigid():275] [14950] Entering Rigid Motion Correction
    82027917 [motion_correction.py:motion_correct_rigid():276] [14950] 0.46466473
    82028375 [movies.py:      extract_shifts():346] [14950] min_val in extract_shifts: -0.18021032929420472
    82028376 [movies.py:      extract_shifts():348] [14950] Movie average is negative. Removing 1st percentile.
    82028535 [movies.py:        apply_shifts():445] [14950] cubic interpolation
    82028654 [movies.py:      extract_shifts():346] [14950] min_val in extract_shifts: -0.18021032929420472
    82028656 [movies.py:      extract_shifts():348] [14950] Movie average is negative. Removing 1st percentile.
    82028823 [movies.py:        apply_shifts():445] [14950] cubic interpolation
    82028944 [movies.py:      extract_shifts():346] [14950] min_val in extract_shifts: -0.18021032929420472
    82028945 [movies.py:      extract_shifts():348] [14950] Movie average is negative. Removing 1st percen

Motion correction has been done!


In [23]:
#%% compare with original movie
display_movie = True
m_els = cm.load(fname_mc)
if display_movie:
    m_orig = cm.load_movie_chain(fnames)
    ds_ratio = 0.2
    cm.concatenate([m_orig.resize(1, 1, ds_ratio) ,
                    m_els.resize(1, 1, ds_ratio)], 
                   axis=2).play(fr=10, gain=1, magnification=3, offset=0)  # press q to exit

  0%|          | 0/1 [00:00<?, ?it/s]    82234372 [movies.py:                load():1500] [14950] loading in memory
    82234500 [movies.py:                load():1503] [14950] mmap
100%|██████████| 1/1 [00:00<00:00, 11.88it/s]
    82234693 [movies.py:              resize():958] [14950] reshaping along z
    82234763 [movies.py:              resize():958] [14950] reshaping along z


In [25]:
print (fname_mc)
print (fname_new)

['/home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/sample_concat_rig__d1_235_d2_265_d3_1_order_F_frames_667_.mmap']
/home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/memmap__d1_235_d2_265_d3_1_order_C_frames_667_.mmap


In [27]:
# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')

In [30]:
# parameters for source extraction and deconvolution

#Tune these parameters!!!!
min_corr = .8       # min peak value from correlation image
min_pnr = 5        # min peak to noise ratio from PNR image
gSig = (3, 3)       # gaussian width of a 2D gaussian kernel, which approximates a neuron
gSiz = (13, 13)     # average diameter of a neuron, in general 4*gSig+1

p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None
Ain = None          # possibility to seed with predetermined binary masks
merge_thr = .7      # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 1            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = 0             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
ssub_B = 1          # additional downsampling factor in space for background
ring_size_factor = 1.2  # radius of ring is gSiz*ring_size_factor

opts.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thr': merge_thr,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': bord_px})                # number of pixels to not consider in the borders)

    82974228 [params.py:                 set():858] [14950] Changing key rf in group patch from None to 40
    82974228 [params.py:                 set():858] [14950] Changing key stride in group patch from None to 20
    82974229 [params.py:                 set():858] [14950] Changing key nb_patch in group patch from 1 to 0
    82974230 [params.py:                 set():858] [14950] Changing key low_rank_background in group patch from True to None
    82974230 [params.py:                 set():858] [14950] Changing key del_duplicates in group patch from False to True
    82974231 [params.py:                 set():858] [14950] Changing key p in group preprocess from 2 to 1
    82974232 [params.py:                 set():858] [14950] Changing key method_init in group init from greedy_roi to corr_pnr
    82974232 [params.py:                 set():858] [14950] Changing key K in group init from 30 to None
    82974233 [params.py:                 set():858] [14950] Changing key gSig in group

CNMFParams:

data:

{'caiman_version': '1.6.2',
 'decay_time': 0.4,
 'dims': (235, 265),
 'dxy': (1, 1),
 'fnames': ['/home/qiushou/Documents/QS_data/miniscope/12_Miniscope/20190930/191082/H13_M24_S14/sample_concat.avi'],
 'fr': 10,
 'last_commit': 'FILE-1568170642',
 'mmap_C': None,
 'mmap_F': None,
 'var_name_hdf5': 'mov'}

spatial_params:

{'block_size_spat': 5000,
 'dist': 3,
 'expandCore': array([[0, 0, 1, 0, 0],
       [0, 1, 1, 1, 0],
       [1, 1, 1, 1, 1],
       [0, 1, 1, 1, 0],
       [0, 0, 1, 0, 0]]),
 'extract_cc': True,
 'maxthr': 0.1,
 'medw': None,
 'method_exp': 'dilate',
 'method_ls': 'lasso_lars',
 'n_pixels_per_process': None,
 'nb': 0,
 'normalize_yyt_one': True,
 'nrgthr': 0.9999,
 'num_blocks_per_run_spat': 20,
 'se': array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=uint8),
 'ss': array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=uint8),
 'thr_method': 'nrg',
 'update_background_components': True}

temporal_params:

{'ITER': 2,
 'bas_nonne

In [32]:
# compute some summary images (correlation and peak to noise)
cn_filter, pnr = cm.summary_images.correlation_pnr(images[::5], gSig=gSig[0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
# inspect the summary images and set the parameters
inspect_correlation_pnr(cn_filter, pnr)

In [33]:
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
cnm.fit(images)

    83208385 [cluster.py:         stop_server():358] [14950] stop_cluster(): done
    83208457 [params.py:                 set():858] [14950] Changing key n_processes in group patch from 1 to 8
    83208458 [params.py:                 set():858] [14950] Changing key init_batch in group online from 200 to 667
    83208459 [cnmf.py:                 fit():468] [14950] (667, 235, 265)
    83208460 [params.py:                 set():858] [14950] Changing key medw in group spatial from None to (3, 3)
    83208460 [cnmf.py:                 fit():485] [14950] Using 8 processes
    83208462 [params.py:                 set():858] [14950] Changing key n_pixels_per_process in group preprocess from None to 7784
    83208463 [params.py:                 set():858] [14950] Changing key n_pixels_per_process in group spatial from None to 7784
    83208463 [cnmf.py:                 fit():496] [14950] using 7784 pixels per process
    83208464 [cnmf.py:                 fit():497] [14950] using 5000 block_s

    83208543 [map_reduce.py:        cnmf_patches():124] [25921] memmap__d1_235_d2_265_d3_1_order_C_frames_667__LOG_ 14160_33040file loaded
    83208561 [params.py:                 set():858] [25920] Changing key n_processes in group patch from 8 to 1
    83208568 [params.py:                 set():858] [25917] Changing key n_processes in group patch from 8 to 1
    83208570 [params.py:                 set():858] [25918] Changing key n_processes in group patch from 8 to 1
    83208573 [params.py:                 set():858] [25920] Changing key rf in group patch from 40 to None
    83208574 [params.py:                 set():858] [25920] Changing key stride in group patch from 20 to None
    83208575 [params.py:                 set():858] [25920] Changing key p in group preprocess from 1 to 0
    83208576 [params.py:                 set():858] [25920] Changing key p in group temporal from 1 to 0
    83208577 [cnmf.py:                 fit():455] [25920] Parallel processing in a single patch

    83208619 [params.py:                 set():858] [25921] Changing key p in group temporal from 1 to 0
    83208620 [pre_processing.py:interpolate_missing_data():53] [25919] Checking for missing data entries (NaN)
    83208620 [pre_processing.py:interpolate_missing_data():53] [25917] Checking for missing data entries (NaN)
    83208620 [pre_processing.py:interpolate_missing_data():53] [25915] Checking for missing data entries (NaN)
    83208620 [cnmf.py:                 fit():455] [25921] Parallel processing in a single patch is not available for loaded in memory or sliced data.
    83208621 [cnmf.py:                 fit():468] [25921] (667, 81, 81)
    83208627 [pre_processing.py:interpolate_missing_data():53] [25921] Checking for missing data entries (NaN)
    83208623 [cnmf.py:                 fit():485] [25921] Using 1 processes
    83208623 [cnmf.py:                 fit():496] [25921] using 1600 pixels per process
    83208624 [cnmf.py:                 fit():497] [25921] using 5

    83232265 [spatial.py:update_spatial_components():215] [25920] Memory mapping
    83232278 [spatial.py:update_spatial_components():221] [25920] Updating Spatial Components using lasso lars
    83232392 [initialization.py:      greedyROI_corr():1049] [25916] Updating spatial components
    83232414 [spatial.py:update_spatial_components():183] [25916] Computing support of spatial components
    83232424 [initialization.py:      greedyROI_corr():1049] [25915] Updating spatial components
    83232435 [spatial.py:update_spatial_components():183] [25915] Computing support of spatial components
    83232504 [spatial.py:update_spatial_components():215] [25916] Memory mapping
    83232524 [spatial.py:update_spatial_components():221] [25916] Updating Spatial Components using lasso lars
    83232559 [spatial.py:update_spatial_components():215] [25915] Memory mapping
    83232579 [spatial.py:update_spatial_components():221] [25915] Updating Spatial Components using lasso lars
    83232564 [init

    83234538 [temporal.py:    update_iteration():385] [25916] 14 out of total 14 temporal components updated
    83234552 [spatial.py:update_spatial_components():278] [25922] Computing residuals
    83234555 [spatial.py:update_spatial_components():306] [25922] Updating done in 2s
    83234558 [initialization.py:      greedyROI_corr():1069] [25916] Searching for more neurons in the residual
    83234577 [spatial.py:update_spatial_components():309] [25922] Removing created tempfiles
    83234602 [initialization.py:      greedyROI_corr():1056] [25922] Updating temporal components
    83234608 [temporal.py:update_temporal_components():201] [25922] Generating residuals
    83234632 [temporal.py:update_temporal_components():215] [25922] entering the deconvolution 
    83234642 [temporal.py:    update_iteration():385] [25922] 5 out of total 9 temporal components updated
    83234668 [temporal.py:    update_iteration():385] [25922] 8 out of total 9 temporal components updated
    83234683 [tem

...

    83329242 [initialization.py:      greedyROI_corr():1081] [25917] Merging components
    83329245 [merging.py:    merge_components():296] [25917] No more components merged!
    83329249 [initialization.py:      greedyROI_corr():1087] [25917] Updating spatial components
    83329250 [spatial.py:update_spatial_components():183] [25917] Computing support of spatial components
    83329294 [spatial.py:update_spatial_components():215] [25917] Memory mapping
    83329295 [spatial.py:update_spatial_components():221] [25917] Updating Spatial Components using lasso lars
    83329296 [initialization.py:      greedyROI_corr():1049] [25916] Updating spatial components
    83329299 [spatial.py:update_spatial_components():183] [25916] Computing support of spatial components
    83329316 [spatial.py:update_spatial_components():215] [25916] Memory mapping
    83329317 [spatial.py:update_spatial_components():221] [25916] Updating Spatial Components using lasso lars
    83329334 [spatial.py:update_sp

    83331864 [initialization.py:      greedyROI_corr():1095] [25919] Updating temporal components
    83331870 [temporal.py:update_temporal_components():201] [25919] Generating residuals
    83331879 [temporal.py:update_temporal_components():215] [25919] entering the deconvolution 
    83331887 [temporal.py:    update_iteration():385] [25919] 1 out of total 4 temporal components updated
    83331894 [temporal.py:    update_iteration():385] [25919] 3 out of total 4 temporal components updated
    83331901 [temporal.py:    update_iteration():385] [25919] 4 out of total 4 temporal components updated
    83331908 [temporal.py:    update_iteration():385] [25919] 1 out of total 4 temporal components updated
    83331916 [temporal.py:    update_iteration():385] [25919] 3 out of total 4 temporal components updated
    83331921 [temporal.py:    update_iteration():385] [25919] 4 out of total 4 temporal components updated
    83331928 [initialization.py:      greedyROI_corr():1102] [25919] Recomp


    83340614 [initialization.py:      greedyROI_corr():1173] [25916] Returning background as b0 and W
    83340616 [warnings.py:        _showwarnmsg():99] [25916] /home/qiushou/.conda/envs/caiman/lib/python3.7/site-packages/scipy/sparse/dia.py:300: RuntimeWarning: divide by zero encountered in remainder
  c = np.arange(num_rows, dtype=np.intc) - (offsets % max_dim)[:, None]

    83340755 [spatial.py:update_spatial_components():254] [25919] thresholding components
    83340762 [spatial.py:update_spatial_components():278] [25919] Computing residuals
    83340763 [spatial.py:update_spatial_components():306] [25919] Updating done in 0s
    83340763 [spatial.py:update_spatial_components():309] [25919] Removing created tempfiles
    83340766 [initialization.py:      greedyROI_corr():1142] [25919] Updating temporal components
    83340769 [temporal.py:update_temporal_components():201] [25919] Generating residuals
    83340775 [temporal.py:update_temporal_components():215] [25919] entering th

    83688920 [components_evaluation.py: evaluate_components():396] [25916] tB:-2.0,tA:9.0
    83688921 [components_evaluation.py: evaluate_components():396] [25917] tB:-2.0,tA:9.0
    83688921 [components_evaluation.py: evaluate_components():401] [25916] Computing event exceptionality delta
    83688922 [components_evaluation.py: evaluate_components():401] [25917] Computing event exceptionality delta
    83688943 [components_evaluation.py: evaluate_components():405] [25916] Removing Baseline
    83688943 [components_evaluation.py: evaluate_components():405] [25917] Removing Baseline
    83688944 [components_evaluation.py: evaluate_components():426] [25916] binning data ...
    83688944 [components_evaluation.py: evaluate_components():426] [25917] binning data ...
    83688946 [components_evaluation.py: evaluate_components():430] [25917] interpolating data ...
    83688946 [components_evaluation.py: evaluate_components():430] [25916] interpolating data ...
    83688946 [components_evalu

    83779778 [components_evaluation.py: evaluate_components():426] [25916] binning data ...
    83779779 [components_evaluation.py: evaluate_components():430] [25916] interpolating data ...
    83779780 [components_evaluation.py: evaluate_components():431] [25916] (6, 33)
    83779781 [components_evaluation.py: evaluate_components():405] [25917] Removing Baseline
    83779782 [components_evaluation.py: evaluate_components():426] [25917] binning data ...
    83779783 [components_evaluation.py: evaluate_components():430] [25917] interpolating data ...
    83779784 [components_evaluation.py: evaluate_components():439] [25916] Computing event exceptionality
    83779784 [components_evaluation.py: evaluate_components():431] [25917] (6, 50)
    83779791 [components_evaluation.py: evaluate_components():439] [25917] Computing event exceptionality
    83779804 [components_evaluation.py: evaluate_components():443] [25916] Evaluating spatial footprint
    83779822 [components_evaluation.py: evalu

In [38]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

min_SNR = 8            # adaptive way to set threshold on the transient size
r_values_min = 0.8   # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': r_values_min,
                           'use_cnn': False})
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

print(' ***** ')
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))

# accepted components
cnm.estimates.nb_view_components(img=cn_filter, idx=cnm.estimates.idx_components,
                                denoised_color='red', cmap='gray')

    83779756 [components_evaluation.py:estimate_components_quality():696] [14950] Component evaluation in parallel


 ***** 
Number of total components:  83
Number of accepted components:  30


In [39]:
# without background
cnm.estimates.play_movie(images, q_max=99.9, magnification=1,
                                 include_bck=False, gain_res=1,bpx=bord_px)

In [19]:
cm.stop_server(dview=dview)

     1907683 [cluster.py:         stop_server():358] [7922] stop_cluster(): done


In [ ]:
cnm.save(os.path.join(newpath,'result.hdf5'))